In [ ]:
EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    rules = []
    updates = []
    reading_rules = True
    with open(input_file_name, 'r') as f:
        for line in f:
            line = line.replace("\n", "")
            if not line:
                reading_rules = False
                continue
            if reading_rules:
                rules.append([int(s) for s in line.split('|')])
            else:
                updates.append([int(s) for s in line.replace("\n", "").split(',')])
    return rules, updates


In [ ]:
rules, updates = parse_input(EXAMPLE)

In [ ]:
print(rules, updates)

In [ ]:
def build_pages(rules):
    pages = {}
    for [x, y] in rules:
        if y in pages:
            pages[y]["predecessors"].add(x)
        else:
            pages[y] = {"predecessors": set([x])}
    return pages

In [ ]:
pages = build_pages(rules)
print(pages)

In [ ]:
def check_update(update, pages):
    correct = True
    for i in range(len(update)):
        for j in range(len(update)):
            if (
                j < i
                and update[j] in pages and update[i] in pages[update[j]]["predecessors"]
                or j > i
                and update[i] in pages and update[j] in pages[update[i]]["predecessors"]
            ):
                correct = False
                break
            if not correct:
                break
    return correct

In [ ]:
for update in updates:
    print(check_update(update, pages))

In [ ]:
def get_middle_page(update):
    return update[len(update)//2]

In [ ]:
for update in updates:
    if check_update(update, pages):
        print(get_middle_page(update))


In [ ]:
def part_1(input_file_name):
    rules, updates = parse_input(input_file_name)
    pages = build_pages(rules)
    result = 0
    for update in updates:
        if check_update(update, pages):
            result += get_middle_page(update)
    print(result)

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

In [ ]:
def reorder_update(incorrect_update, rules, pages):
    # Rebuild an update with pages in the correct order
    reordered_update = []
    for [x, y] in rules:
        if x in incorrect_update and y in incorrect_update:
            if x not in reordered_update and y not in reordered_update:
                reordered_update.extend([x, y])
            elif x not in reordered_update and y in reordered_update:
                reordered_update.insert(reordered_update.index(y), x)
            elif x in reordered_update and y not in reordered_update:
                reordered_update.insert(reordered_update.index(x)+1, y)
            elif x in reordered_update and y in reordered_update:
                if reordered_update.index(x) < reordered_update.index(y):
                    continue
                # The only complicated case: both pages are already in
                # the list but in the wrong order. So we need to swap them.
                # But we need to be sure to move the pages between them 
                # that need to be moved to stay in order.
                pages_to_move = []
                i = reordered_update.index(y)+1
                end = reordered_update.index(x)
                # Remove the page with the highest index
                reordered_update.pop(reordered_update.index(x))
                while i < end:
                    # If the current page needs to be before the page we're moving,
                    # we need to move the current page as well
                    if x in pages and reordered_update[i] in pages[x]["predecessors"]:
                        pages_to_move.append(reordered_update.pop(i))
                        end -= 1
                    else:
                        i+=1
                pages_to_move.append(x)
                # Now we just insert the pages that needed to move back into the list, at the right index.
                reordered_update = reordered_update[:reordered_update.index(y)] + pages_to_move + reordered_update[reordered_update.index(y):]
    return reordered_update

In [ ]:
for update in updates:
    if not check_update(update, pages):
        print(reorder_update(update, rules, pages))

In [ ]:
def part_2(input_file_name):
    rules, updates = parse_input(input_file_name)
    pages = build_pages(rules)
    result = 0
    for update in updates:
        if not check_update(update, pages):
            result += get_middle_page(reorder_update(update, rules, pages))
    print(result)

In [ ]:
part_2(EXAMPLE)

In [ ]:
part_2(INPUT)